In [1]:
import tensorflow_model_optimization as tfmot

In [2]:
import tensorflow as tf

In [3]:
(X_train,y_train),(X_test,y_test) = tf.keras.datasets.mnist.load_data()

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(28,28)),
    tf.keras.layers.Reshape((28,28,1)),
    tf.keras.layers.Conv2D(12,3, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])

Metal device set to: Apple M1


2022-10-25 20:40:22.654100: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-25 20:40:22.654178: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [6]:
model.fit(X_train,y_train,epochs=5)

Epoch 1/5


2022-10-25 20:40:23.244459: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-10-25 20:40:23.438857: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1875/1875 [==============================] - 9s 4ms/step - loss: 1.4065
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0989
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0894
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0893
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0884


In [8]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

In [23]:
sp_model = tfmot.sparsity.keras.strip_pruning(model) # gzip 압축 방식을 이용해서

In [26]:
import tempfile

In [27]:
_,x = tempfile.mkstemp('.h5')

In [28]:
x

'/var/folders/9k/jsf_2t1d6ts48d1mpfj3nxp00000gn/T/tmpy5wq79qq.h5'

In [29]:
tf.keras.models.save_model(sp_model, x)

In [24]:
sp_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 12)        120       
                                                                 
 flatten (Flatten)           (None, 8112)              0         
                                                                 
 dense (Dense)               (None, 10)                81130     
                                                                 
Total params: 81,250
Trainable params: 81,250
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.weights == sp_model.weights

True

In [9]:
# Compute end step to finish pruning after 2 epochs.
batch_size = 100
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = X_train.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

In [10]:
model_for_pruning = prune_low_magnitude(model, tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step))

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_reshape  (None, 28, 28, 1)        1         
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_conv2d   (None, 26, 26, 12)       230       
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_flatten  (None, 8112)             1         
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_dense (  (None, 10)               162252    
 PruneLowMagnitude)                                              
                                                                 
Total params: 162,484
Trainable params: 81,250
Non-train

In [11]:
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
#   tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(X_train, y_train,
                  batch_size=batch_size, epochs=epochs, validation_split=0.1,
                  callbacks=callbacks)

Epoch 1/2


2022-10-25 20:41:00.726480: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


535/540 [============================>.] - ETA: 0s - loss: 0.0753 - accuracy: 0.9786

2022-10-25 20:41:07.170415: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


540/540 [==============================] - 7s 11ms/step - loss: 0.0751 - accuracy: 0.9787 - val_loss: 0.0509 - val_accuracy: 0.9852
Epoch 2/2
540/540 [==============================] - 5s 10ms/step - loss: 0.0491 - accuracy: 0.9860 - val_loss: 0.0500 - val_accuracy: 0.9857


# another

In [12]:
import numpy as np
batch_size = 100
epochs = 2
end = np.ceil(len(X_train) / 100).astype('int32')*epochs

pruing_model = tfmot.sparsity.keras.prune_low_magnitude(model, 
                                                        tfmot.sparsity.keras.PolynomialDecay(0.5,0.8,0,end))

In [13]:
#pruning_model = tfmot.sparsity.keras.prune_low_magnitude(model, 
#                                                         tfmot.sparsity.keras.ConstantSparsity(0.3,0))

In [14]:
pruing_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [15]:
tfmot.sparsity.keras.UpdatePruningStep()

In [21]:
pruing_model.fit(X_train,y_train,batch_size=batch_size, epochs=epochs,
                           validation_split=0.1,
                           callbacks=[tfmot.sparsity.keras.UpdatePruningStep()])

Epoch 1/2
540/540 [==============================] - 5s 9ms/step - loss: 0.0277 - val_loss: 0.0862
Epoch 2/2
540/540 [==============================] - 5s 9ms/step - loss: 0.0255 - val_loss: 0.0852


In [17]:
pruing_model.weights

[<tf.Variable 'prune_low_magnitude_reshape/pruning_step:0' shape=() dtype=int64, numpy=1080>,
 <tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 12) dtype=float32, numpy=
 array([[[[-5.1327208e-03, -1.1846285e-02, -3.9335336e-03,
           -7.2680144e-03, -4.9629370e-03, -5.9615854e-02,
           -2.3391540e-03, -2.2966003e-03, -1.9713464e-01,
           -1.6139152e-02,  9.5978336e-05,  1.6244840e-02]],
 
         [[-1.4457966e-02, -1.3228089e-01, -5.0334182e-02,
           -5.1631760e-02, -1.7990516e-02,  4.7749855e-02,
           -1.8566605e-01, -8.2268743e-03, -1.1302059e-01,
           -1.5423664e-01, -2.2817396e-02,  3.3499731e-03]],
 
         [[-7.7534705e-02, -2.4050175e-01, -1.8971395e-01,
            1.8528284e-04,  4.1256320e-02, -2.4180973e-02,
           -1.0658428e-02, -1.2907846e-01, -5.6280620e-02,
           -5.1148009e-02, -2.5290828e-02, -2.0946564e-02]]],
 
 
        [[[-1.1890536e-02,  1.3192016e-02,  3.7245315e-02,
           -2.6644149e-03, -5.6417964e-02, -3.0160

In [22]:
pruing_model.weights

[<tf.Variable 'prune_low_magnitude_reshape/pruning_step:0' shape=() dtype=int64, numpy=3240>,
 <tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 12) dtype=float32, numpy=
 array([[[[-2.9907102e-04, -7.1060313e-03, -2.7330576e-03,
           -7.2680144e-03, -3.0363677e-03, -6.0931619e-02,
           -3.1223763e-03, -2.2966003e-03, -1.9713464e-01,
           -1.3803040e-02,  4.0584430e-03,  1.7325846e-02]],
 
         [[-7.1631871e-02, -1.3005792e-01, -6.4557076e-02,
           -5.1631760e-02, -1.9265585e-02,  4.8503555e-02,
           -1.5557157e-01, -8.2268743e-03, -1.1302059e-01,
           -1.5855007e-01, -2.7892206e-02,  5.4725879e-03]],
 
         [[-4.9324580e-02, -2.9582143e-01, -2.1046680e-01,
            1.8528284e-04,  4.3177519e-02, -2.8437953e-02,
           -3.4237111e-03, -1.2907846e-01, -5.6280620e-02,
           -5.5110373e-02, -3.5336066e-02, -1.1766943e-02]]],
 
 
        [[[-2.7417533e-02,  2.1629114e-02,  4.1128911e-02,
           -2.6644149e-03, -7.0218734e-02, -1.9016

### tensorflow

In [ ]:
tf.keras32 : 

In [30]:
%timeit 1.+1.

4.09 ns ± 0.00167 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [31]:
%timeit 1+1

4.09 ns ± 0.00377 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)
